In [11]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict

In [16]:
url = "https://hvl.instructure.com/api/v1/accounts/1/terms"

dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
        dataliste = []
        for element in data['enrollment_terms']:
            dataliste.append(element)
        df = pd.DataFrame(dataliste)
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei same:
alle_terminar = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)

https://hvl.instructure.com/api/v1/accounts/1/terms?page=2&per_page=100


In [23]:
terminar = alle_terminar[(alle_terminar['name'].str.contains(r'20[0-9]{2} VÅR'))|(alle_terminar['name'].str.contains(r'20[0-9]{2} HØST'))]

In [24]:
terminar

,id,name,start_at,end_at,created_at,workflow_state,grading_period_group_id,sis_term_id,sis_import_id
1,2,2017 HØST,None,None,2017-03-29T19:49:36Z,active,None,2017-HØST,NaN
2,6,2017 HØST,None,None,2017-11-18T01:01:34Z,active,None,2017_HØST,14402.0
3,7,2017 VÅR,None,None,2017-12-05T18:19:16Z,active,None,2017_VÅR,6373.0
4,8,2018 VÅR,None,None,2017-12-05T18:19:16Z,active,None,2018_VÅR,14402.0
5,12,2017 HØST-2018 VÅR,None,None,2018-02-16T01:02:00Z,active,None,2017_HØST-2018_VÅR,14402.0
...,...,...,...,...,...,...,...,...,...
115,356,2026 VÅR-2027 VÅR,None,None,2025-04-01T07:18:07Z,active,None,2026_VÅR-2027_VÅR,29017.0
116,357,2025 HØST-2026 HØST,None,None,2025-04-01T07:18:07Z,active,None,2025_HØST-2026_HØST,29017.0
117,358,2025 HØST-2029 VÅR,None,None,2025-04-01T07:18:07Z,active,None,2025_HØST-2029_VÅR,29017.0
118,359,2026 VÅR-2029 HØST,None,None,2025-04-01T07:18:07Z,active,None,2026_VÅR-2029_HØST,29017.0


In [15]:
alledata.to_csv('terminar.csv', index=False)

In [39]:
def start(rad):
    if '-' in rad['name']:
        start = rad['name'].split('-')[0].strip()
    else:
        start = rad['name']
    if "HØST" in start:
        start = start.replace("HØST", "SISTE")
    else:
        start = start.replace("VÅR", "FØRSTE")
    return start


def slutt(rad):
    if '-' in rad['name']:
        slutt = rad['name'].split('-')[1].strip()
    else:
        slutt = rad['name']
    if "HØST" in slutt:
        slutt = slutt.replace("HØST", "SISTE")
    else:
        slutt = slutt.replace("VÅR", "FØRSTE")
    return slutt


terminar['start'] = terminar.apply(start, axis=1)
terminar['slutt'] = terminar.apply(slutt, axis=1)

C:\Users\AKV\AppData\Local\Temp\ipykernel_22568\961785125.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  terminar['start'] = terminar.apply(start, axis=1)
C:\Users\AKV\AppData\Local\Temp\ipykernel_22568\961785125.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  terminar['slutt'] = terminar.apply(slutt, axis=1)


In [40]:
terminar

,id,name,start_at,end_at,created_at,workflow_state,grading_period_group_id,sis_term_id,sis_import_id,start,slutt
1,2,2017 HØST,None,None,2017-03-29T19:49:36Z,active,None,2017-HØST,NaN,2017 SISTE,2017 SISTE
2,6,2017 HØST,None,None,2017-11-18T01:01:34Z,active,None,2017_HØST,14402.0,2017 SISTE,2017 SISTE
3,7,2017 VÅR,None,None,2017-12-05T18:19:16Z,active,None,2017_VÅR,6373.0,2017 FØRSTE,2017 FØRSTE
4,8,2018 VÅR,None,None,2017-12-05T18:19:16Z,active,None,2018_VÅR,14402.0,2018 FØRSTE,2018 FØRSTE
5,12,2017 HØST-2018 VÅR,None,None,2018-02-16T01:02:00Z,active,None,2017_HØST-2018_VÅR,14402.0,2017 SISTE,2018 FØRSTE
...,...,...,...,...,...,...,...,...,...,...,...
115,356,2026 VÅR-2027 VÅR,None,None,2025-04-01T07:18:07Z,active,None,2026_VÅR-2027_VÅR,29017.0,2026 FØRSTE,2027 FØRSTE
116,357,2025 HØST-2026 HØST,None,None,2025-04-01T07:18:07Z,active,None,2025_HØST-2026_HØST,29017.0,2025 SISTE,2026 SISTE
117,358,2025 HØST-2029 VÅR,None,None,2025-04-01T07:18:07Z,active,None,2025_HØST-2029_VÅR,29017.0,2025 SISTE,2029 FØRSTE
118,359,2026 VÅR-2029 HØST,None,None,2025-04-01T07:18:07Z,active,None,2026_VÅR-2029_HØST,29017.0,2026 FØRSTE,2029 SISTE


In [46]:
eldre = terminar[terminar['slutt'] <= "2023 SISTE"]
len(eldre)

66

In [49]:
avslutt_semestre = eldre['id'].tolist()

In [50]:
type(avslutt_semestre)

list

In [51]:
avslutt_semestre    

[2,
 6,
 7,
 8,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 39,
 40,
 41,
 73,
 74,
 75,
 76,
 77,
 79,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 90,
 91,
 92,
 93,
 196,
 197,
 198,
 199,
 200,
 202,
 203,
 204,
 205,
 208,
 209,
 210,
 212,
 215,
 220,
 11]